# Velr + Polars Showcase 

This notebook demonstrates a realistic **Polars → cleanup → Velr graph → export → analytics/plots** pipeline in Cypher.

**What you'll see**
- Import & clean raw tabular data with **Polars**
- Bind cleaned tables into **Velr** and build a graph (Users, Products, BOUGHT/RETURNED)
- Compute co-purchases, top-K recommendations, coupon lift, retention cohorts
- Export results to **Polars**/**Pandas** and make plots with **matplotlib**

> Data: a synthetic retail-like dataset saved as `orders_synthetic.csv`.

In [ ]:
%pip install velr --force-reinstall
%pip install pandas polars pyarrow matplotlib --quiet

In [ ]:
from IPython.display import display
import polars as pl
import pandas as pd
from velr.driver import Velr

CSV_PATH = "../data/orders_synthetic.csv"  # adjust if needed
print("Imports OK")

In [ ]:
# 0) Open in-memory Velr
db = Velr.open(None)
print("DB ready")

In [ ]:
# 1) Load CSV with Polars
raw = pl.read_csv(CSV_PATH, try_parse_dates=True)
raw.head()

In [ ]:

# 2) Clean & normalize (version-agnostic sessionization)


clean = (
    raw
    .with_columns(
        pl.col("user").str.strip_chars().str.to_lowercase(),
        pl.col("product").str.strip_chars().str.to_lowercase(),
        pl.col("category").str.strip_chars().str.to_lowercase(),
        (pl.col("qty") * pl.col("unit_price")).round(2).alias("line_total"),
    )
    .sort(["user","ts"])
    # new session if gap > 3 days for each user; no .dt.*, just duration comparison
    .with_columns(
        (
            (pl.col("ts").diff().over("user") > pl.duration(days=3))
            .fill_null(False)            # first row per user
            .cast(pl.Int64)
        ).alias("new_sess_flag")
    )
)

# Polars API compatibility: cumsum name differs on older versions
try:
    clean = clean.with_columns(
        pl.col("new_sess_flag").cumsum().over("user").alias("session_id")
    )
except Exception:
    clean = clean.with_columns(
        pl.col("new_sess_flag").cum_sum().over("user").alias("session_id")
    )

# Deduplicate and tidy
clean = clean.drop(["new_sess_flag"]).unique(subset=["order_id","user","product"])
clean.head()


In [ ]:
# 3) Bind to Velr and create graph
users_pl    = clean.select("user").unique()
products_pl = clean.select("product","category").unique()
buys_pl     = clean.select("user","product","ts","qty","line_total","coupon_applied","session_id")

FRAC = 0.05
n_ret = max(1, int((getattr(buys_pl, "height", len(buys_pl))) * FRAC))

# Make ~5% returned
returns_pl = buys_pl.sample(fraction=FRAC, with_replacement=False, seed=1)

db.bind_polars("_users_pl", users_pl)
db.bind_polars("_products_pl", products_pl)
db.bind_polars("_buys_pl", buys_pl)
db.bind_polars("_returns_pl", returns_pl)

db.run("""
UNWIND BIND('_users_pl') AS r
CREATE (:User {id:r.user});
""")

db.run("""
UNWIND BIND('_products_pl') AS r
CREATE (:Product {id:r.product, category:r.category});
""")

db.run("""
UNWIND BIND('_buys_pl') AS r
MATCH (u:User {id:r.user}),(p:Product {id:r.product})
CREATE (u)-[:BOUGHT {ts:r.ts, qty:r.qty, total:r.line_total, coupon:r.coupon_applied, session:r.session_id}]->(p);
""")

db.run("""
UNWIND BIND('_returns_pl') AS r
MATCH (u:User {id:r.user})-[:BOUGHT {ts:r.ts, session:r.session_id}]->(p:Product {id:r.product})
CREATE (u)-[:RETURNED {ts:r.ts, session:r.session_id}]->(p);
""")

print("Graph created")

In [ ]:
# 4A) Top products by distinct buyers (no WITH)
top_products_pl = db.to_polars("""
MATCH (p:Product)<-[:BOUGHT]-(:User)
RETURN p.id AS product, count(*) AS buyers
ORDER BY buyers DESC, product
LIMIT 20
""")
top_products_pl

In [ ]:
# 4B) User out-degree
deg_pd = db.to_pandas("""
MATCH (u:User)-[:BOUGHT]->(:Product)
RETURN u.id AS user, count(*) AS deg
ORDER BY deg DESC, user
""")
deg_pd.head()

In [ ]:
# 4C) Product co-purchases (pairs) and Top-K per product (WITH-less using Polars)
pairs_pl = db.to_polars("""
MATCH (u:User)-[:BOUGHT]->(p1:Product), (u)-[:BOUGHT]->(p2:Product)
WHERE p1.id < p2.id
RETURN p1.id AS prod_a, p2.id AS prod_b, count(*) AS weight
""")

left  = pairs_pl.select(
    pl.col("prod_a").alias("src"),
    pl.col("prod_b").alias("dst"),
    pl.col("weight"),
)
right = pairs_pl.select(
    pl.col("prod_b").alias("src"),
    pl.col("prod_a").alias("dst"),
    pl.col("weight"),
)

symm = pl.concat([left, right])  # same columns, same order
# Top-K per src
K = 5
symm = (
    symm
    .sort(["src","weight"], descending=[False, True])
    .group_by("src")
    .head(K)
)
symm.head(20)

In [ ]:
# 4D) Category coupon lift
agg_pl = db.to_polars("""
MATCH (:User)-[b:BOUGHT]->(p:Product)
RETURN p.category AS category, b.coupon AS coupon, sum(b.total) AS revenue
""")

lift = (
    agg_pl
    .pivot(values="revenue", index="category", columns="coupon")
    .fill_null(0)
    .with_columns((pl.col("true")/(pl.col("false")+1e-9)).alias("coupon_lift"))
    .sort("coupon_lift", descending=True)
)
lift

In [ ]:
# 5) Plots
import matplotlib.pyplot as plt

# Degree distribution
deg_dist = (
    deg_pd["deg"]
    .value_counts(sort=False)
    .sort_index()
    .rename_axis("deg")
    .reset_index(name="users_with_deg")
)

plt.figure()
plt.bar(deg_dist["deg"].values, deg_dist["users_with_deg"].values)
plt.title("User out-degree distribution (# of distinct products bought)")
plt.xlabel("out-degree (deg)")
plt.ylabel("# users with this deg")
plt.tight_layout()
plt.show()

